In [35]:
!pip install transformers==4.10.0

  Using cached transformers-4.10.0-py3-none-any.whl.metadata (51 kB)
  Using cached tokenizers-0.10.3-cp39-cp39-win_amd64.whl.metadata (5.8 kB)
Using cached transformers-4.10.0-py3-none-any.whl (2.8 MB)
Using cached tokenizers-0.10.3-cp39-cp39-win_amd64.whl (2.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.0.0 requires transformers<5.0.0,>=4.34.0, but you have transformers 4.10.0 which is incompatible.


In [1]:
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]


In [2]:
pip show transformers

Name: transformersNote: you may need to restart the kernel to use updated packages.

Version: 4.10.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: d:\anaconda\envs\py39\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, sacremoses, tokenizers, tqdm
Required-by: sentence-transformers


In [1]:
import pandas as pd
import numpy as np
import re
import demoji

# Load dataset
df = pd.read_csv("D:\\Sinophobia Sentiment Analysis\\archive\\Tweets_Australia\\Tweets_Australia.csv", low_memory=False)


In [2]:
df

,Unnamed: 0,created_at,id,text,user_location
0,11,Wed Dec 08 04:25:46 +0000 2021,1468436616263909377,RT @AaronSiriSG: Litigation update: \n\nFDA d...,"Iasi, Romania"
1,229,Wed Dec 08 04:25:57 +0000 2021,1468436661633630209,RT @covidbaseau: The TGA has approved the Mode...,Canberra
2,236,Wed Dec 08 04:26:06 +0000 2021,1468436701043314692,RT @SimoLove: Federal Health Minister Greg Hun...,"Wurundjeri Richmond, Melbourne"
3,250,Wed Dec 08 04:26:07 +0000 2021,1468436704755286020,@twityouse Are these the deniers that are wear...,"Victoria, Australia"
4,257,Wed Dec 08 04:26:07 +0000 2021,1468436706424545281,RT @drsimonegold: WOW: a preprint study for Th...,Perth WA
...,...,...,...,...,...
3212459,3404933,1314784000636985344,Sat Oct 10 04:25:07 +0000 2020,RT @michellebrasier: I BOUGHT A PILATES REFORM...,"Sydney, Australia"
3212460,3404947,1314784000846692352,Sat Oct 10 04:25:07 +0000 2020,RT @livcaisley: Exclusive: The aged-care watch...,"Wollongong, New South Wales"
3212461,3404956,1314783999412371456,Sat Oct 10 04:25:07 +0000 2020,RT @DiamondandSilk: The commission canceled th...,"Richmond, TX"
3212462,3405043,1314784010002788354,Sat Oct 10 04:25:10 +0000 2020,@mjrowland68 You live in a city where the medi...,Sydney


In [3]:
# Data cleaning
df = df.drop(columns=['Unnamed: 0', 'id', 'user_location'])

df = df.dropna(axis=0)


In [4]:
df

,created_at,text
0,Wed Dec 08 04:25:46 +0000 2021,RT @AaronSiriSG: Litigation update: \n\nFDA d...
1,Wed Dec 08 04:25:57 +0000 2021,RT @covidbaseau: The TGA has approved the Mode...
2,Wed Dec 08 04:26:06 +0000 2021,RT @SimoLove: Federal Health Minister Greg Hun...
3,Wed Dec 08 04:26:07 +0000 2021,@twityouse Are these the deniers that are wear...
4,Wed Dec 08 04:26:07 +0000 2021,RT @drsimonegold: WOW: a preprint study for Th...
...,...,...
3212459,1314784000636985344,RT @michellebrasier: I BOUGHT A PILATES REFORM...
3212460,1314784000846692352,RT @livcaisley: Exclusive: The aged-care watch...
3212461,1314783999412371456,RT @DiamondandSilk: The commission canceled th...
3212462,1314784010002788354,@mjrowland68 You live in a city where the medi...


In [5]:
import pandas as pd

# Initialize an empty list to hold DataFrames
dfs_to_concat = []

# List of sinophobic terms
sinophobic_terms = ['china', 'chinese', 'sinophobia','sinophobic', 'prc', 'wuhan', 'hubei', 'beijing', 'kung flu', 'chn','cn', 'ccp', 'yellow peril', 'chink', 'chinks', 
                    'chingchong', 'ching chong', 'gook', 'chyna', 'mainland', 'mainlander', 'bugland', 'chines', 
                    'mainla', 'chinazi', 'bugmen', 'chankoro', 'insectoid']

for term in sinophobic_terms:
    filtered_df = df[df['text'].str.contains(f'\\b{term}\\b', case=False, na=False, regex=True)]
    print(f"Term: {term}, Count: {len(filtered_df)}")
    print(filtered_df['text'].head(5))  # Print the first 5 rows to inspect the content
    dfs_to_concat.append(filtered_df)


# Concatenate all filtered DataFrames
final = pd.concat(dfs_to_concat, ignore_index=True)

# Remove duplicate entries
final = final.drop_duplicates(subset=['text'])

# Reset the index of the final DataFrame
final = final.reset_index(drop=True)

# Display the first few rows of the final DataFrame
final.head()



Term: china, Count: 12458
17     China’s #yuan advanced to the strongest level ...
112    RT @jsolomonReports: Fauci’s agency funded Chi...
558    @globaltimesnews And yet China has yet to apol...
679    BBC News - Covid: Vaccines should work against...
979    These MF'rs want to lecture China on human rig...
Name: text, dtype: object
Term: chinese, Count: 5729
2606    RT @songpinganq: @dk82976510 I don't know exac...
2621    RT @songpinganq: @Ev1HasAnOpinion @imajollyrog...
2627    RT @songpinganq: @Ev1HasAnOpinion @imajollyrog...
2635    RT @songpinganq: @AdeldMeyer @Bluewre73334583 ...
2669    Malaya Chinese on Lime, Don't miss to visit - ...
Name: text, dtype: object
Term: sinophobia, Count: 11
510473     Sinophobia is the undercurrent of the anti vax...
527166     RT @hermit_hwarang: People continue to not get...
1322334    Amazing line-up @grantwyeth writing about the ...
1326895    RT @PulbuRonn: It seems that the wokes' cry fo...
1410248    RT @CarlZha: Media ghouls who peddled

,created_at,text
0,Wed Dec 08 04:26:39 +0000 2021,China’s #yuan advanced to the strongest level ...
1,Wed Dec 08 04:30:07 +0000 2021,RT @jsolomonReports: Fauci’s agency funded Chi...
2,Wed Dec 08 04:49:55 +0000 2021,@globaltimesnews And yet China has yet to apol...
3,Wed Dec 08 04:56:32 +0000 2021,BBC News - Covid: Vaccines should work against...
4,Wed Dec 08 05:07:41 +0000 2021,These MF'rs want to lecture China on human rig...


In [6]:
len(final)

14120

In [7]:
final.to_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\AusAfterFilter.csv")

In [2]:
final = pd.read_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\AusAfterFilter.csv")

In [3]:
final

,Unnamed: 0,created_at,text
0,0,Wed Dec 08 04:26:39 +0000 2021,China’s #yuan advanced to the strongest level ...
1,1,Wed Dec 08 04:30:07 +0000 2021,RT @jsolomonReports: Fauci’s agency funded Chi...
2,2,Wed Dec 08 04:49:55 +0000 2021,@globaltimesnews And yet China has yet to apol...
3,3,Wed Dec 08 04:56:32 +0000 2021,BBC News - Covid: Vaccines should work against...
4,4,Wed Dec 08 05:07:41 +0000 2021,These MF'rs want to lecture China on human rig...
...,...,...,...
14115,14115,Fri Nov 05 08:21:53 +0000 2021,RT @VictorQ09111737: @aislinge @abcnews we jus...
14116,14116,1342376510993252352,RT @hildabast: Mice vaccinated with a Covid va...
14117,14117,1353933769565069312,RT @myknittingwool: @pallisier @JudyLee8104675...
14118,14118,Wed Sep 08 21:55:54 +0000 2021,RT @WestenderNews: Those commentating on the Q...


In [4]:
final = final.drop(columns=['Unnamed: 0'])

In [5]:
df_withtime = final

In [6]:
final = final.drop(columns=['created_at'])

In [7]:
final.head()

,text
0,China’s #yuan advanced to the strongest level ...
1,RT @jsolomonReports: Fauci’s agency funded Chi...
2,@globaltimesnews And yet China has yet to apol...
3,BBC News - Covid: Vaccines should work against...
4,These MF'rs want to lecture China on human rig...


In [8]:
contractionsWithAnotherInvertedComma = {
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [9]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [10]:
abbreviations_dict = {
    "ccp": "chinese communist party", "cn": "china", "chn": "china", "prc": "peoples republic of china",
    "btw": "by the way", "brb": "be right back",
    "b4n": "bye for now",
    "fyi": "for your information",
    "ilu": "i love you",
    "iow": "in other words",
    "roflol": "rolling on the floor laughing out loud",
    "rotflmao": "rolling on the floor laughing my ass off",
    "fb": "facebook", "ig": "instagram", "li": "linkedin", "yt": "youtube", "tw": "twitter",
    "dm": "direct message", "mt": "modified tweet", "pm": "private message", "rt": "retweet",
    "b2b": "business to business", "b2c": "business to consumer", "cmgr": "community manager",
    "cms": "content management system", "cpc": "cost per click", "cpm": "cost per thousand impressions",
    "cr": "conversion rate", "cro": "conversion rate optimization", "cta": "call to action",
    "ctr": "click-through rate", "roi": "return on investment", "smb": "small and midsize businesses",
    "smp": "social media platform", "smm": "social media marketing", "smo": "social media optimization",
    "solomo": "social, local and mobile", "srp": "social relationship platform", "tos": "terms of service",
    "ugc": "user-generated content", "api": "application programming interface", "cx": "customer experience",
    "esp": "email service provider", "ga": "google analytics", "isp": "internet service provider",
    "pv": "page views", "rss": "really simple syndication", "saas": "software as a service",
    "sem": "search engine marketing", "seo": "search engine optimization", "sov": "share of voice",
    "ui": "user interface", "url": "uniform resource locator", "uv": "unique views", "ux": "user experience",
    "afaik": "as far as i know", "ama": "ask me anything", "btaim": "be that as it may",
    "bts": "behind the scenes", "dae": "does anyone else", "dyk": "did you know",
    "eli5": "explain like i’m five", "fbf": "flashback friday", "fbo": "facebook official",
    "ff": "follow friday", "fomo": "fear of missing out", "ftfy": "fixed that for you",
    "ftw": "for the win", "g2g": "got to go", "gg": "good game", "gtr": "got to run",
    "hbd": "happy birthday", "hifw": "how i feel when", "hmb": "hit me back", "hmu": "hit me up",
    "ht": "hat tip", "h/t": "hat tip", "hth": "here to help", "icymi": "in case you missed it",
    "idc": "i don’t care", "idk": "i don’t know", "ikr": "i know, right?", "ily": "i love you",
    "imho": "in my humble opinion", "imo": "in my opinion", "irl": "in real life", "jk": "just kidding",
    "lmao": "laughing my ass off", "lmk": "let me know", "lms": "like my status",
    "lol": "laughing out loud", "mcm": "man crush monday", "mfw": "my face when",
    "mtfbwy": "may the force be with you", "nbd": "no big deal", "nm": "not much",
    "nsfw": "not safe for work", "nvm": "never mind", "oh": "overheard", "omw": "on my way",
    "ootd": "outfit of the day", "op": "original poster", "otp": "one true pairing", "ppl": "people",
    "rofl": "rolling on the floor laughing", "roflmao": "rolling on the floor laughing my ass off",
    "sfw": "safe for work", "smh": "shaking my head", "tbh": "to be honest", "tbbh": "to be brutally honest",
    "tbt": "throwback thursday", "tfw": "that feeling when", "tgif": "thank god it’s friday",
    "til": "today i learned", "tl;dr": "too long; didn’t read", "tmi": "too much information",
    "wbu": "what about you?", "wbw": "way back wednesday", "wfh": "work from home", "yolo": "you only live once",
    "afk": "away from keyboard", "asap": "as soon as possible", "atk": "at the keyboard",
    "atm": "at the moment", "a3": "anytime, anywhere, anyplace", "bak": "back at keyboard",
    "bbl": "be back later", "bbs": "be back soon", "bfn": "bye for now", "brt": "be right there",
    "b4": "before", "cu": "see you", "cul8r": "see you later", "cya": "see you", "faq": "frequently asked questions",
    "fc": "fingers crossed", "fwiw": "for what it's worth", "gal": "get a life", "gn": "good night",
    "gmta": "great minds think alike", "gr8": "great!", "g9": "genius", "ic": "i see", "icq": "i seek you",
    "kiss": "keep it simple, stupid", "ldr": "long distance relationship", "ltns": "long time no see",
    "l8r": "later", "mte": "my thoughts exactly", "m8": "mate", "nrn": "no reply necessary", "oic": "oh i see",
    "pita": "pain in the a..", "prt": "party", "prw": "parents are watching", "qpsa": "que pasa?",
    "sk8": "skate", "stats": "your sex and age", "asl": "age, sex, location", "thx": "thank you",
    "ttfn": "ta-ta for now!", "ttyl": "talk to you later", "u": "you", "u2": "you too", "u4e": "yours for ever",
    "wb": "welcome back", "wtf": "what the f...", "wtg": "way to go!", "wuf": "where are you from?",
    "w8": "wait...", "7k": "sick:-d laugher"
}


In [11]:
# Preprocessing
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions

    def lower(self, tweet):
        return tweet.lower()

    def abbreviate(self, tweet):
        tweet_tokens = tweet.split(' ')
        for i, token in enumerate(tweet_tokens):
            cleaned_token = re.sub('[^a-zA-Z0-9-_.]', '', token)
            if cleaned_token.lower() in abbreviations_dict:
                tweet_tokens[i] = abbreviations_dict[cleaned_token.lower()]
        return ' '.join(tweet_tokens)

    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet

    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])

        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)

    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)

    def remove_url(self, tweet):
        return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags = re.MULTILINE)

    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_url(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)

        return tweet

In [12]:
import csv
import string
pp_class = preprocess(final, contractions, contractionsWithAnotherInvertedComma)
final['text'] = final['text'].apply(lambda x : pp_class.preprocess_tweet(x))

In [13]:
def bruteGen(tweet):
    tweet = tweet.replace("indiavscorona", "india versus coronavirus")
    tweet = tweet.replace("outbreakindia", "outbreak india")
    tweet = tweet.replace("real”", "real")
    tweet = tweet.replace("mutra", "urine")
    tweet = tweet.replace("fakenews", "fake news")
    tweet = tweet.replace("“omg", "oh my god")
    tweet = tweet.replace("“damn", "damn")
    tweet = tweet.replace("god’s", "gods")
    tweet = tweet.replace("lockdownextension", "lockdown extension")
    tweet = tweet.replace("कोरोना", "coronavirus")
    tweet = tweet.replace("indiathanks", "india thanks")
    tweet = tweet.replace("coronacoronavirus", "coronavirus")
    tweet = tweet.replace('coronavirusinsa', "coronavirus in south africa")
    tweet = tweet.replace('coronaviruscanada', 'coronavirus canada')
    tweet = tweet.replace('coronavirusau', 'coronavirus australia')
    tweet = tweet.replace('coronavirusaus', 'coronavirus australia')
    tweet = tweet.replace('cuomoprimetime', 'new york governor prime time')
    tweet = tweet.replace('letsfightcoronavirus', 'let us fight coronavirus')
    tweet = tweet.replace("covid 19", "coronavirus")
    tweet = tweet.replace("covid19", "coronavirus")
    tweet = tweet.replace("covid", "coronavirus")
    tweet = tweet.replace("aprilfoolsday", "april fools day")
    tweet = tweet.replace("covid-19", "coronavirus")
    tweet = tweet.replace("stayathome", "stay at home")
    tweet = tweet.replace("“april", "april")
    tweet = tweet.replace("“i", "i")
    tweet = tweet.replace("aprilfools", "april fools")
    tweet = tweet.replace("coronavirusoutbreak", "coronavirus outbreak")
    tweet = tweet.replace("virus-19", "coronavirus")
    tweet = tweet.replace("fool’s", "fools")
    tweet = tweet.replace("what’s", "what is")
    tweet = tweet.replace("coronavirus”", "coronavirus")
    tweet = tweet.replace("fools”", "fools")
    tweet = tweet.replace("stayhome", "stay home")
    tweet = tweet.replace("quarantinelife", "quarantine life")
    tweet = tweet.replace("tablighijamaat", "muslims")
    tweet = tweet.replace("corona”", "coronavirus")
    tweet = tweet.replace("fauci", "physician")
    tweet = tweet.replace("april’s", "april")
    tweet = tweet.replace("pmkcallscurfewextension", "prime minister calls for curfew extension")
    tweet = tweet.replace("“virus", "coronavirus")
    tweet = tweet.replace("virus”", "coronavirus")
    tweet = tweet.replace("“corona", "coronavirus")
    tweet = tweet.replace("coronavirustruth", "coronavirus truth")
    tweet = tweet.replace("socialdistancing", "social distancing")
    tweet = tweet.replace("homestaysafe", "home stay safe")
    tweet = tweet.replace("“coronavirus", "coronavirus")
    tweet = tweet.replace("coronavirusupdate", "coronavirus update")
    tweet = tweet.replace("virusvirus", "coronavirus")
    tweet = tweet.replace("coronaviruspandemic", "coronavirus pandemic")
    tweet = tweet.replace("thelockdown", "the lockdown")
    tweet = tweet.replace("nizamuddin", "delhi")
    tweet = tweet.replace("trump’s", "donald trump")
    tweet = tweet.replace("“the", "the")
    tweet = tweet.replace("virus2019", "coronavirus")
    tweet = tweet.replace("indiafightscorona", "india fights coronavirus")
    tweet = tweet.replace("homesavelives", "home save lives")
    tweet = tweet.replace("everyone’s", "everyone")
    tweet = tweet.replace("coronariskforprisoners", "coronavirus risk for prisoners")
    tweet = tweet.replace("coronavirususa", "coronavirus usa")
    tweet = tweet.replace("tablighi", "mosque")
    tweet = tweet.replace("delhimarkaz", "delhi mosque")
    tweet = tweet.replace("coronajihad", "coronavirus struggle")
    tweet = tweet.replace("coronajihaad", "coronavirus struggle")
    tweet = tweet.replace("aprilfool", "april fool")
    tweet = tweet.replace("trumppressconference", "trump press conference")
    tweet = tweet.replace("i’m", "i am")
    tweet = tweet.replace("tigerking", "tiger king")
    tweet = tweet.replace("it’s", "it is")
    tweet = tweet.replace("trumpvirus", "trump virus")
    tweet = tweet.replace("today’s", "today is")
    tweet = tweet.replace("“you", "you")
    tweet = tweet.replace("“a", "a")
    tweet = tweet.replace("fools’", "fools")
    tweet = tweet.replace("rtgnews", "news")
    tweet = tweet.replace("19india", "india")
    tweet = tweet.replace("coronavirusindia", "coronavirus india")
    tweet = tweet.replace("y’all", "you all")
    tweet = tweet.replace("मीडिया", "media")
    tweet = tweet.replace("here’s", "here is")
    tweet = tweet.replace("“we", "we")
    tweet = tweet.replace("“fuck", "fuck")
    tweet = tweet.replace("flattenthecurve", "flatten the curve")
    tweet = tweet.replace("jammuandkashmir", "jammu and kashmir")
    tweet = tweet.replace("chriscuomo", "new york governor")
    tweet = tweet.replace("‘april", "april")
    tweet = tweet.replace("dranbumani", "doctor")
    tweet = tweet.replace("tndemandsmasstesting", "tamil nadu demands mass testing")
    tweet = tweet.replace("tabligi", "muslims")
    tweet = tweet.replace("don’t", "do not")
    tweet = tweet.replace("वायरस", "virus")
    tweet = tweet.replace("letsfightvirus", "let us fight virus")
    return tweet


In [14]:
final['text'] = final['text'].apply(lambda x : bruteGen(x))

In [15]:
final.head

<bound method NDFrame.head of                                                     text
0      china’s  yuan advanced to the strongest level ...
1      retweet  physician’s agency funded china’s bat...
2       and yet china has yet to apologise for this f...
3      bbc news   coronavirus  vaccines should work a...
4      these mf rs want to lecture china on human rig...
...                                                  ...
14115  retweet    we just had all our rights stripped...
14116  retweet  mice vaccinated with a coronavirus va...
14117                          retweet           chines…
14118  retweet  those commentating on the qld  vaccin...
14119  retweet  democrats  bugmen  are comfortable wi...

[14120 rows x 1 columns]>

In [16]:
final.to_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\AusAfterpreprocess.csv")

In [17]:
labels = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Joking']

In [18]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [19]:
df = pd.DataFrame()
df['Tweet'] = final['text']
values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * len(final)
df[labels] = values

In [20]:
df.reset_index(drop=True, inplace=True)
df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,china’s yuan advanced to the strongest level ...,0,0,0,0,0,0,0,0,0,0
1,retweet physician’s agency funded china’s bat...,0,0,0,0,0,0,0,0,0,0
2,and yet china has yet to apologise for this f...,0,0,0,0,0,0,0,0,0,0
3,bbc news coronavirus vaccines should work a...,0,0,0,0,0,0,0,0,0,0
4,these mf rs want to lecture china on human rig...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
14115,retweet we just had all our rights stripped...,0,0,0,0,0,0,0,0,0,0
14116,retweet mice vaccinated with a coronavirus va...,0,0,0,0,0,0,0,0,0,0
14117,retweet chines…,0,0,0,0,0,0,0,0,0,0
14118,retweet those commentating on the qld vaccin...,0,0,0,0,0,0,0,0,0,0


In [21]:
import torch
class CustomDataset(torch.utils.data.Dataset):
   def __init__(self,df,tokenizer,max_len):
     self.df = df
     self.tokenizer = tokenizer
     self.max_len = max_len
     self.title = self.df['Tweet']
     self.targets = self.df[labels].values

   def __len__(self):
     return len(self.title)

   def __getitem__(self,index):
     title = str(self.title[index])
     title = " ".join(title.split())

     inputs = self.tokenizer.encode_plus(
         title,
         None,
         add_special_tokens=True,
         max_length=self.max_len,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )
     return {
         'input_ids':inputs['input_ids'].flatten(),
         'attention_mask':inputs['attention_mask'].flatten(),
         'token_type_ids':inputs['token_type_ids'].flatten(),
         'targets':torch.FloatTensor(self.targets[index])
     }

In [22]:
MAX_LEN=256
_dataset = CustomDataset(df, tokenizer, MAX_LEN)
_data_loader = torch.utils.data.DataLoader(
    _dataset,
    shuffle=False,
    batch_size=8,
    num_workers=0
)

In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [24]:
class BERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = torch.nn.Dropout(0.3)
        self.layer3 = torch.nn.Linear(768, 10)

    def forward(self, ids, mask, token_type_ids, return_dict = False):
        unw, out_1 = self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[0], self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final


model = BERT()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (layer1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [25]:
model = torch.load("D:\\Sinophobia Sentiment Analysis\\Transformers4.10Bertmodel2.pth")

In [26]:
model.eval()
bert_outputs = []
with torch.no_grad():

  for index, batch in enumerate(_data_loader):
         input_ids = batch['input_ids'].to(device, dtype=torch.long)
         attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
         token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
         targets =  batch['targets'].to(device, dtype=torch.float)
         outputs = model(input_ids, attention_mask, token_type_ids)
         bert_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

In [27]:
test_outputs = np.array(bert_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [28]:
df[labels] = test_outputs.astype(int)

In [29]:
df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,china’s yuan advanced to the strongest level ...,0,0,0,0,0,0,0,0,1,0
1,retweet physician’s agency funded china’s bat...,0,0,0,0,0,0,0,1,1,0
2,and yet china has yet to apologise for this f...,0,0,0,0,0,0,1,1,0,0
3,bbc news coronavirus vaccines should work a...,0,0,0,0,0,0,1,1,0,0
4,these mf rs want to lecture china on human rig...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
14115,retweet we just had all our rights stripped...,0,0,0,0,0,0,1,0,0,0
14116,retweet mice vaccinated with a coronavirus va...,1,0,0,0,0,0,0,0,0,1
14117,retweet chines…,0,0,0,0,0,0,0,0,0,1
14118,retweet those commentating on the qld vaccin...,0,0,0,0,0,0,1,0,0,0


In [30]:
df_withtime.reset_index(drop=True, inplace=True)
df_withtime

,created_at,text
0,Wed Dec 08 04:26:39 +0000 2021,China’s #yuan advanced to the strongest level ...
1,Wed Dec 08 04:30:07 +0000 2021,RT @jsolomonReports: Fauci’s agency funded Chi...
2,Wed Dec 08 04:49:55 +0000 2021,@globaltimesnews And yet China has yet to apol...
3,Wed Dec 08 04:56:32 +0000 2021,BBC News - Covid: Vaccines should work against...
4,Wed Dec 08 05:07:41 +0000 2021,These MF'rs want to lecture China on human rig...
...,...,...
14115,Fri Nov 05 08:21:53 +0000 2021,RT @VictorQ09111737: @aislinge @abcnews we jus...
14116,1342376510993252352,RT @hildabast: Mice vaccinated with a Covid va...
14117,1353933769565069312,RT @myknittingwool: @pallisier @JudyLee8104675...
14118,Wed Sep 08 21:55:54 +0000 2021,RT @WestenderNews: Those commentating on the Q...


In [31]:
Aus = pd.DataFrame()
Aus['Tweet'] = df['Tweet']
Aus['text'] = df_withtime['text']
Aus['date'] = df_withtime['created_at']
Aus[labels] = df[labels]
Aus

,Tweet,text,date,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,china’s yuan advanced to the strongest level ...,China’s #yuan advanced to the strongest level ...,Wed Dec 08 04:26:39 +0000 2021,0,0,0,0,0,0,0,0,1,0
1,retweet physician’s agency funded china’s bat...,RT @jsolomonReports: Fauci’s agency funded Chi...,Wed Dec 08 04:30:07 +0000 2021,0,0,0,0,0,0,0,1,1,0
2,and yet china has yet to apologise for this f...,@globaltimesnews And yet China has yet to apol...,Wed Dec 08 04:49:55 +0000 2021,0,0,0,0,0,0,1,1,0,0
3,bbc news coronavirus vaccines should work a...,BBC News - Covid: Vaccines should work against...,Wed Dec 08 04:56:32 +0000 2021,0,0,0,0,0,0,1,1,0,0
4,these mf rs want to lecture china on human rig...,These MF'rs want to lecture China on human rig...,Wed Dec 08 05:07:41 +0000 2021,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14115,retweet we just had all our rights stripped...,RT @VictorQ09111737: @aislinge @abcnews we jus...,Fri Nov 05 08:21:53 +0000 2021,0,0,0,0,0,0,1,0,0,0
14116,retweet mice vaccinated with a coronavirus va...,RT @hildabast: Mice vaccinated with a Covid va...,1342376510993252352,1,0,0,0,0,0,0,0,0,1
14117,retweet chines…,RT @myknittingwool: @pallisier @JudyLee8104675...,1353933769565069312,0,0,0,0,0,0,0,0,0,1
14118,retweet those commentating on the qld vaccin...,RT @WestenderNews: Those commentating on the Q...,Wed Sep 08 21:55:54 +0000 2021,0,0,0,0,0,0,1,0,0,0


In [32]:
Aus.to_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\Result\\Aus.csv")